In [4]:
%pylab inline
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy

Populating the interactive namespace from numpy and matplotlib


In [5]:
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [6]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)

In [7]:
train_df = pd.read_json('/Users/Home/Desktop/CMPT 459/MI2/final_train_df.json')

In [15]:
from sklearn.metrics.pairwise import manhattan_distances
downtown = [40.7209, -74.0007]

def man_distances(row):
    dist = manhattan_distances([[row.latitude, row.longitude]], [downtown])
    return dist[0][0]


train_df['dt_distance'] = train_df.apply(lambda row: man_distances(row), axis=1)
train_df.dt_distance.head()

0    0.0569
1    0.0589
2    0.0748
3    0.0646
4    0.0494
Name: dt_distance, dtype: float64

In [16]:
v1_selected_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "pet_policy", "listing_type", 
                     "unit_features", "building_features", "apartment", "appliance", "bathroom", "bedroom",
                     "building", "contact", "floor", "high", "kitchen", "room", "num_photos", 
                     "description_len", "mall_distance", "dt_distance"]

In [20]:
target_num_map = {'high':0, 'medium':1, 'low':2}
train_df['int_level'] = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

In [43]:
X = train_df[v1_selected_feats].values
y = train_df["int_level"]

In [19]:
import xgboost as xgb

In [24]:
from sklearn.model_selection import KFold

In [45]:
param = {
    'objective': 'multi:softprob',  
    'eta': 0.3, #learning rate, default is 0.3
    'max_depth': 3,  #default is 6, increase this value = more complex model
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'min_child_weight': 1, # default is 1, larger value = more conservative model
    'subsample': 1, # default is 1, smaller == less overfitting
    'colsample_bytree': 1, # default is 1
    'seed': 42
    }

param0 = {
    'objective': 'multi:softprob',  
    'eta': 0.1, 
    'max_depth': 3,  #default is 6, increase this value = more complex model
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'min_child_weight': 1, # default is 1, larger value = more conservative model
    'subsample': 0.6, # default is 1, smaller == less overfitting
    'colsample_bytree': 0.6, # default is 1
    'seed': 42
    }

param1 = {
    'objective': 'multi:softprob',  
    'eta': 0.1, 
    'max_depth': 4,  #default is 6, increase this value = more complex model
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'min_child_weight': 1, # default is 1, larger value = more conservative model
    'subsample': 0.7, # default is 1, smaller == less overfitting
    'colsample_bytree': 0.7, # default is 1
    'seed': 42
    }

param2 = {
    'objective': 'multi:softprob',  
    'eta': 0.1, 
    'max_depth': 5,  #default is 6, increase this value = more complex model
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'min_child_weight': 1, # default is 1, larger value = more conservative model
    'subsample': 0.8, # default is 1, smaller == less overfitting
    'colsample_bytree': 0.8, # default is 1
    'seed': 42
    }

param3 = {
    'objective': 'multi:softprob',  
    'eta': 0.1, 
    'max_depth': 6,  #default is 6, increase this value = more complex model
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'min_child_weight': 1, # default is 1, larger value = more conservative model
    'subsample': 0.9, # default is 1, smaller == less overfitting
    'colsample_bytree': 0.9, # default is 1
    'seed': 42
    }

param4 = {
    'objective': 'multi:softprob',  
    'eta': 0.1, #
    'max_depth': 7,  #default is 6, increase this value = more complex model
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'min_child_weight': 1, # default is 1, larger value = more conservative model
    'subsample': 1, # default is 1, smaller == less overfitting
    'colsample_bytree': 1, # default is 1
    'seed': 42
    }
# param['objective'] = 'multi:softprob'
# param['eta'] = 0.1
# param['max_depth'] = 6
# param['num_class'] = 3
# param['eval_metric'] = "mlogloss"
# param['min_child_weight'] = 1
# param['subsample'] = 0.7
# param['colsample_bytree'] = 0.7
# param['seed'] = seed_val

In [46]:
param_list = [dict() for x in range(5)]
param_list[0] = param0
param_list[1] = param1
param_list[2] = param2
param_list[3] = param3
param_list[4] = param4
# param_list[1]

In [ ]:
# D_train = xgb.DMatrix(X_train, label=y_train)
# D_test = xgb.DMatrix(X_test, label=y_test)
# xgtest = xgb.DMatrix(test_X)"

In [47]:
test_scores = []
train_scores = []
log_scores = []

# step_arr = [50, 100, 150, 200, 500]
steps = 500
i = 0
c = KFold(n_splits=5, random_state=42, shuffle=True)


for train_index, test_index in c.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)

    watchlist = [ (dtrain,'train'), (dvalid, 'test') ]
    model = xgb.train(param_list[i], dtrain, steps, watchlist, early_stopping_rounds=20)
    i += 1

#     y_val = model.predict(dvalid)
    
    
#     test_scores.append(model.score(X_test, y_test))
#     train_scores.append(clf.score(X_train, y_train))
#     y_test_pred = clf.predict_proba(X_test)
#     log_scores.append(log_loss(y_test, y_test_pred))

[0]	train-mlogloss:1.04657	test-mlogloss:1.0462
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.00177	test-mlogloss:1.00105
[2]	train-mlogloss:0.964557	test-mlogloss:0.963603
[3]	train-mlogloss:0.931553	test-mlogloss:0.930139
[4]	train-mlogloss:0.903146	test-mlogloss:0.90176
[5]	train-mlogloss:0.879953	test-mlogloss:0.878407
[6]	train-mlogloss:0.858609	test-mlogloss:0.856831
[7]	train-mlogloss:0.841201	test-mlogloss:0.839296
[8]	train-mlogloss:0.82448	test-mlogloss:0.822533
[9]	train-mlogloss:0.8119	test-mlogloss:0.809863
[10]	train-mlogloss:0.798996	test-mlogloss:0.796738
[11]	train-mlogloss:0.788248	test-mlogloss:0.786117
[12]	train-mlogloss:0.777396	test-mlogloss:0.775189
[13]	train-mlogloss:0.768149	test-mlogloss:0.765974
[14]	train-mlogloss:0.759975	test-mlogloss:0.757852
[15]	train-mlogloss:0.752903	test-mlogloss:0.750709
[16]	train-mlogloss:0.746557	test-ml

[155]	train-mlogloss:0.618032	test-mlogloss:0.627535
[156]	train-mlogloss:0.617731	test-mlogloss:0.627303
[157]	train-mlogloss:0.617439	test-mlogloss:0.62708
[158]	train-mlogloss:0.61726	test-mlogloss:0.62704
[159]	train-mlogloss:0.616973	test-mlogloss:0.626891
[160]	train-mlogloss:0.616624	test-mlogloss:0.626608
[161]	train-mlogloss:0.616394	test-mlogloss:0.626527
[162]	train-mlogloss:0.616145	test-mlogloss:0.626386
[163]	train-mlogloss:0.615864	test-mlogloss:0.626246
[164]	train-mlogloss:0.615513	test-mlogloss:0.625894
[165]	train-mlogloss:0.615199	test-mlogloss:0.625686
[166]	train-mlogloss:0.614907	test-mlogloss:0.625534
[167]	train-mlogloss:0.614717	test-mlogloss:0.625447
[168]	train-mlogloss:0.614513	test-mlogloss:0.625304
[169]	train-mlogloss:0.614182	test-mlogloss:0.625072
[170]	train-mlogloss:0.614014	test-mlogloss:0.624946
[171]	train-mlogloss:0.613695	test-mlogloss:0.624713
[172]	train-mlogloss:0.61342	test-mlogloss:0.624454
[173]	train-mlogloss:0.613114	test-mlogloss:0.6242

[311]	train-mlogloss:0.586337	test-mlogloss:0.60982
[312]	train-mlogloss:0.586167	test-mlogloss:0.609751
[313]	train-mlogloss:0.586018	test-mlogloss:0.609692
[314]	train-mlogloss:0.585791	test-mlogloss:0.609534
[315]	train-mlogloss:0.585648	test-mlogloss:0.609445
[316]	train-mlogloss:0.585479	test-mlogloss:0.609366
[317]	train-mlogloss:0.585358	test-mlogloss:0.609369
[318]	train-mlogloss:0.585172	test-mlogloss:0.609243
[319]	train-mlogloss:0.585061	test-mlogloss:0.609214
[320]	train-mlogloss:0.584913	test-mlogloss:0.609194
[321]	train-mlogloss:0.584737	test-mlogloss:0.609089
[322]	train-mlogloss:0.584597	test-mlogloss:0.609031
[323]	train-mlogloss:0.584441	test-mlogloss:0.608952
[324]	train-mlogloss:0.584277	test-mlogloss:0.608837
[325]	train-mlogloss:0.584151	test-mlogloss:0.608783
[326]	train-mlogloss:0.583976	test-mlogloss:0.608707
[327]	train-mlogloss:0.583861	test-mlogloss:0.608611
[328]	train-mlogloss:0.583714	test-mlogloss:0.608618
[329]	train-mlogloss:0.583545	test-mlogloss:0.6

[467]	train-mlogloss:0.565433	test-mlogloss:0.603552
[468]	train-mlogloss:0.56534	test-mlogloss:0.603493
[469]	train-mlogloss:0.565191	test-mlogloss:0.603499
[470]	train-mlogloss:0.565053	test-mlogloss:0.603456
[471]	train-mlogloss:0.564916	test-mlogloss:0.603417
[472]	train-mlogloss:0.56481	test-mlogloss:0.603367
[473]	train-mlogloss:0.56471	test-mlogloss:0.603396
[474]	train-mlogloss:0.564564	test-mlogloss:0.603345
[475]	train-mlogloss:0.564442	test-mlogloss:0.603368
[476]	train-mlogloss:0.564337	test-mlogloss:0.603354
[477]	train-mlogloss:0.564241	test-mlogloss:0.603351
[478]	train-mlogloss:0.564131	test-mlogloss:0.603324
[479]	train-mlogloss:0.564025	test-mlogloss:0.603334
[480]	train-mlogloss:0.563834	test-mlogloss:0.603261
[481]	train-mlogloss:0.563744	test-mlogloss:0.603261
[482]	train-mlogloss:0.563596	test-mlogloss:0.603215
[483]	train-mlogloss:0.56348	test-mlogloss:0.60315
[484]	train-mlogloss:0.563373	test-mlogloss:0.603098
[485]	train-mlogloss:0.563216	test-mlogloss:0.60314

[122]	train-mlogloss:0.595432	test-mlogloss:0.623586
[123]	train-mlogloss:0.595012	test-mlogloss:0.623359
[124]	train-mlogloss:0.594589	test-mlogloss:0.62306
[125]	train-mlogloss:0.594092	test-mlogloss:0.622894
[126]	train-mlogloss:0.593554	test-mlogloss:0.622582
[127]	train-mlogloss:0.593144	test-mlogloss:0.622401
[128]	train-mlogloss:0.592762	test-mlogloss:0.622351
[129]	train-mlogloss:0.592399	test-mlogloss:0.622199
[130]	train-mlogloss:0.591907	test-mlogloss:0.621872
[131]	train-mlogloss:0.591598	test-mlogloss:0.621716
[132]	train-mlogloss:0.591261	test-mlogloss:0.621591
[133]	train-mlogloss:0.590764	test-mlogloss:0.621446
[134]	train-mlogloss:0.590333	test-mlogloss:0.621179
[135]	train-mlogloss:0.58986	test-mlogloss:0.620993
[136]	train-mlogloss:0.589492	test-mlogloss:0.620883
[137]	train-mlogloss:0.589064	test-mlogloss:0.620709
[138]	train-mlogloss:0.588699	test-mlogloss:0.620567
[139]	train-mlogloss:0.588263	test-mlogloss:0.620376
[140]	train-mlogloss:0.587777	test-mlogloss:0.62

...

[89]	train-mlogloss:0.581848	test-mlogloss:0.622503
[90]	train-mlogloss:0.580946	test-mlogloss:0.622032
[91]	train-mlogloss:0.58028	test-mlogloss:0.621765
[92]	train-mlogloss:0.579452	test-mlogloss:0.621505
[93]	train-mlogloss:0.57881	test-mlogloss:0.621295
[94]	train-mlogloss:0.578033	test-mlogloss:0.620943
[95]	train-mlogloss:0.57748	test-mlogloss:0.6207
[96]	train-mlogloss:0.576767	test-mlogloss:0.620476
[97]	train-mlogloss:0.575898	test-mlogloss:0.619976
[98]	train-mlogloss:0.575209	test-mlogloss:0.619642
[99]	train-mlogloss:0.574753	test-mlogloss:0.619463
[100]	train-mlogloss:0.573963	test-mlogloss:0.619176
[101]	train-mlogloss:0.573386	test-mlogloss:0.618936
[102]	train-mlogloss:0.572726	test-mlogloss:0.618658
[103]	train-mlogloss:0.572125	test-mlogloss:0.618494
[104]	train-mlogloss:0.571351	test-mlogloss:0.618242
[105]	train-mlogloss:0.570611	test-mlogloss:0.618057
[106]	train-mlogloss:0.569809	test-mlogloss:0.617676
[107]	train-mlogloss:0.569161	test-mlogloss:0.617482
[108]	tra

[245]	train-mlogloss:0.5055	test-mlogloss:0.604082
[246]	train-mlogloss:0.505102	test-mlogloss:0.604084
[247]	train-mlogloss:0.504738	test-mlogloss:0.603943
[248]	train-mlogloss:0.50439	test-mlogloss:0.604024
[249]	train-mlogloss:0.504063	test-mlogloss:0.603949
[250]	train-mlogloss:0.503752	test-mlogloss:0.603909
[251]	train-mlogloss:0.503382	test-mlogloss:0.603831
[252]	train-mlogloss:0.503052	test-mlogloss:0.603798
[253]	train-mlogloss:0.502743	test-mlogloss:0.603816
[254]	train-mlogloss:0.50239	test-mlogloss:0.603833
[255]	train-mlogloss:0.502135	test-mlogloss:0.603802
[256]	train-mlogloss:0.501744	test-mlogloss:0.603741
[257]	train-mlogloss:0.501377	test-mlogloss:0.603629
[258]	train-mlogloss:0.50101	test-mlogloss:0.603601
[259]	train-mlogloss:0.500591	test-mlogloss:0.603603
[260]	train-mlogloss:0.500297	test-mlogloss:0.603577
[261]	train-mlogloss:0.499968	test-mlogloss:0.603595
[262]	train-mlogloss:0.499622	test-mlogloss:0.603625
[263]	train-mlogloss:0.499199	test-mlogloss:0.60367

[54]	train-mlogloss:0.580502	test-mlogloss:0.639476
[55]	train-mlogloss:0.579347	test-mlogloss:0.639147
[56]	train-mlogloss:0.578074	test-mlogloss:0.638598
[57]	train-mlogloss:0.576624	test-mlogloss:0.637959
[58]	train-mlogloss:0.575258	test-mlogloss:0.637174
[59]	train-mlogloss:0.573711	test-mlogloss:0.636592
[60]	train-mlogloss:0.572411	test-mlogloss:0.6363
[61]	train-mlogloss:0.571396	test-mlogloss:0.635869
[62]	train-mlogloss:0.569844	test-mlogloss:0.635148
[63]	train-mlogloss:0.56867	test-mlogloss:0.634859
[64]	train-mlogloss:0.567097	test-mlogloss:0.634234
[65]	train-mlogloss:0.566063	test-mlogloss:0.633697
[66]	train-mlogloss:0.56468	test-mlogloss:0.633006
[67]	train-mlogloss:0.563904	test-mlogloss:0.632611
[68]	train-mlogloss:0.562699	test-mlogloss:0.632246
[69]	train-mlogloss:0.561623	test-mlogloss:0.631841
[70]	train-mlogloss:0.560558	test-mlogloss:0.631589
[71]	train-mlogloss:0.559564	test-mlogloss:0.631134
[72]	train-mlogloss:0.558496	test-mlogloss:0.630808
[73]	train-mlogl

[211]	train-mlogloss:0.45957	test-mlogloss:0.615101
[212]	train-mlogloss:0.459156	test-mlogloss:0.615146
[213]	train-mlogloss:0.458723	test-mlogloss:0.615089
[214]	train-mlogloss:0.458138	test-mlogloss:0.615065
[215]	train-mlogloss:0.457821	test-mlogloss:0.61505
[216]	train-mlogloss:0.45739	test-mlogloss:0.615078
[217]	train-mlogloss:0.456903	test-mlogloss:0.615116
[218]	train-mlogloss:0.456198	test-mlogloss:0.615154
[219]	train-mlogloss:0.45571	test-mlogloss:0.615083
[220]	train-mlogloss:0.455183	test-mlogloss:0.615108
[221]	train-mlogloss:0.454564	test-mlogloss:0.615177
[222]	train-mlogloss:0.454171	test-mlogloss:0.615057
[223]	train-mlogloss:0.453747	test-mlogloss:0.61509
[224]	train-mlogloss:0.453227	test-mlogloss:0.615046
[225]	train-mlogloss:0.452572	test-mlogloss:0.615026
[226]	train-mlogloss:0.452111	test-mlogloss:0.615089
[227]	train-mlogloss:0.451756	test-mlogloss:0.615121
[228]	train-mlogloss:0.451392	test-mlogloss:0.615119
[229]	train-mlogloss:0.450887	test-mlogloss:0.61516

[81]	train-mlogloss:0.513876	test-mlogloss:0.606843
[82]	train-mlogloss:0.513134	test-mlogloss:0.606563
[83]	train-mlogloss:0.512065	test-mlogloss:0.606334
[84]	train-mlogloss:0.51069	test-mlogloss:0.606033
[85]	train-mlogloss:0.509516	test-mlogloss:0.605783
[86]	train-mlogloss:0.508802	test-mlogloss:0.605659
[87]	train-mlogloss:0.507887	test-mlogloss:0.6055
[88]	train-mlogloss:0.507052	test-mlogloss:0.605337
[89]	train-mlogloss:0.506315	test-mlogloss:0.605138
[90]	train-mlogloss:0.505338	test-mlogloss:0.604958
[91]	train-mlogloss:0.50391	test-mlogloss:0.604569
[92]	train-mlogloss:0.5032	test-mlogloss:0.604484
[93]	train-mlogloss:0.502464	test-mlogloss:0.604348
[94]	train-mlogloss:0.50136	test-mlogloss:0.604127
[95]	train-mlogloss:0.500199	test-mlogloss:0.604057
[96]	train-mlogloss:0.498946	test-mlogloss:0.603692
[97]	train-mlogloss:0.497647	test-mlogloss:0.60351
[98]	train-mlogloss:0.496632	test-mlogloss:0.603446
[99]	train-mlogloss:0.496013	test-mlogloss:0.603377
[100]	train-mlogloss

In [48]:
full_X = train_df[v1_selected_feats].values
full_y = train_df["int_level"]

In [49]:
dtrain = xgb.DMatrix(full_X, label=full_y)

In [50]:
model = xgb.train(param_list[0], dtrain, steps)

In [51]:
test_df = pd.read_json('/Users/Home/Desktop/CMPT 459/MI2/final_test_df_m3.json')

test_X = test_df[v1_selected_feats].values
xgtest = xgb.DMatrix(test_X)


In [52]:
preds = model.predict(xgtest)

In [53]:
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("xgb_v1.csv", index=False)